In [2]:
#Libraries
import torch
import cv2
from PIL import Image
%pip install -r requirements.txt 
from torchvision.transforms import functional as F
import tkinter as tk
from tkinter import ttk
from tkinter import messagebox
from PIL import ImageTk, Image
from PIL import ImageTk, Image
from torchvision import transforms
import os
import openpyxl
import requests





Defaulting to user installation because normal site-packages is not writeable
Note: you may need to restart the kernel to use updated packages.


ERROR: Could not open requirements file: [Errno 2] No such file or directory: 'requirements.txt'


In [3]:
def read_images_from_folder(folder_path):
    images = []
    for filename in os.listdir(folder_path):
        if filename.endswith(".jpg") or filename.endswith(".png"):
            image_path = os.path.join(folder_path, filename)
            images.append(image_path)
            if len(images) == 3:
                break
    return images



In [4]:
# Define the license plate detection function
def detect_license_plate(image_path, config_path, weights_path, confidence_threshold=0.5):
    # Load the YOLOv5 model
    model = torch.hub.load('C:\\Users\\Home\\Desktop\\yolov5test\\yolov5', 'custom', path=weights_path, source='local')
    model.yaml = config_path

    # Load the input image
    img = Image.open(image_path)
    #Applying image preprocessing
    image = cv2.cvtColor(cv2.imread(image_path), cv2.COLOR_BGR2GRAY)
    image = cv2.medianBlur(image, 5)  # Apply median blur
    image = cv2.GaussianBlur(image, (5, 5), 0)  # Apply Gaussian blur

    # Run license plate detection on the preprocessed image
    results = model(image)

    # Get the bounding box coordinates and confidence scores for the detected license plates
    pred = results.pred[0]
    boxes = pred[:, :4].detach().numpy()
    confidences = pred[:, 4].detach().numpy()
    # Iterate over the license plate detections
    for box, confidence in zip(boxes, confidences):
        if confidence > confidence_threshold:
            # Draw the bounding box on the image
            x_min, y_min, x_max, y_max = box.astype(int)
            cv2.rectangle(image, (x_min, y_min), (x_max, y_max), (0, 255, 0), 2)
            # Crop the license plate region
            license_plate = img.crop((x_min, y_min, x_max, y_max))
            # Write the confidence score on the license plate image
            label = f"Confidence: {confidence:.2f}"
            cv2.putText(image, label, (x_min, y_min - 10), cv2.FONT_HERSHEY_SIMPLEX, 0.9, (0, 255, 0), 2)
            # Save the cropped license plate image
            license_plate.save("cropped_plate.jpg")
    # Return the cropped license plate image
    return Image.open("cropped_plate.jpg")

In [5]:
def OCR_Character_recognition(image):
    model_id = '344f4216-a780-4b5f-b366-1d53e62fa05b'
    api_key = 'd5382ff4-008d-11ee-9688-5ae95f6e1ac0'

    # Save the cropped image locally
    image.save('cropped_image.jpg')

    # Prepare the API request
    url = 'https://app.nanonets.com/api/v2/ObjectDetection/Model/' + model_id + '/LabelFile/'
    data = {'file': open('cropped_image.jpg', 'rb'), 'modelId': ('', model_id)}
    auth = requests.auth.HTTPBasicAuth(api_key, '')

    # Send the API request
    response = requests.post(url, auth=auth, files=data)

    # Parse the JSON response
    res = response.json()

    # Check if 'prediction' exists and is not empty
    if 'result' in res and len(res['result']) > 0 and 'prediction' in res['result'][0] and len(res['result'][0]['prediction']) > 0:
        ocr_text = res['result'][0]['prediction'][0]['ocr_text']
    else:
        ocr_text = "OCR Text not found"
        
    return ocr_text


In [6]:
#Integrating the ALPR system with the Database sheet function to retrieve Vehicle's Information
def search_vehicle_information(license_plate):
    # Load the Excel workbook
    workbook = openpyxl.load_workbook('C:\\Users\\Home\\Desktop\\yolov5test\\ALPR_Model\\Database.xlsx')
    # Get the active sheet in the workbook
    sheet = workbook.active

    # Iterate over the rows in the sheet starting from the second row (skipping the header)
    for row in sheet.iter_rows(min_row=2, values_only=True):
        # Check if the license plate in the current row matches the given license_plate
        if row[0] == license_plate:
            # If a match is found, return a dictionary containing the information for that license plate
            return {
                'Plate Number': row[0],
                'Color': row[1],
                'Manufacturer': row[2],
                'Model': row[3],
                'Manufacture Year': row[4],
                'Owner\'s name': row[5],
                'License Expiry date': row[9],
                'Expiry License - Update Required': row[11]
            }

    # If no match is found, return None
    return None


In [7]:
#Integrating the ALPR system with the Database sheet function to retrieve Owner's Information
def search_owner_information(license_plate):
    # Load the Excel workbook
    workbook = openpyxl.load_workbook('C:\\Users\\Home\\Desktop\\yolov5test\\ALPR_Model\\Database.xlsx')
    # Get the active sheet in the workbook
    sheet = workbook.active

    # Iterate over the rows in the sheet starting from the second row (skipping the header)
    for row in sheet.iter_rows(min_row=2, values_only=True):
        # Check if the license plate in the current row matches the given license_plate
        if row[0] == license_plate:
            # If a match is found, return a dictionary containing the information for that license plate
            return {
                'Owner\'s name': row[5],
                'Owner\'s ID': row[6],
                'Birthday date': row[7],
                'Phone Number ': row[8],
                'Address': row[10]
            }

    # If no match is found, return None
    return None


In [8]:
# function to check if the username and password are correct then creates the second GUI ,third GUI pages
def check_login():
    ## Get the entered username and password from the entry fields
    username = entry_username.get()
    password = entry_password.get()
    ## Check if the entered username and password are correct
    if username == "admin" and password == "12345":
        # if the username and password are correct, show a message box with a success message
        messagebox.showinfo("Login Successful", "You have successfully logged in!")

        #Create a new window for the second GUI:License Plate Recognition page
        window2 = tk.Toplevel()
        window2.title("License Plate Recognition")
        window2.geometry("800x600")
        
        #New function for creating third GUI page for displaying Vehicle Information that are retrieved by Integrating the ALPR with database :Integrated Database
        #open_window3()function will be executed when the "Check" button is clicked. 
        def open_window3(license_plate, info_type):
           ## Create a new window for displaying information
           window3 = tk.Toplevel()
           window3.title("Integrated Database")
           window3.geometry("800x600")
           
           # Retrieve the row from the excel sheet file by calling (search_license_plate function) which takes (license_plate) number that was entered by the user
           row = search_vehicle_information(license_plate)
           if row:
              # Create a label for displaying information
              info_label = tk.Label(window3, text="Vehicle's Information", font=("Arial", 12))
              info_label.pack()

              # Create a frame for displaying the information in a table format
              table_frame = tk.Frame(window3)
              table_frame.pack()

              # Check if the "Expiry License - Update Required" field is "Expired"
              if row['Expiry License - Update Required'] == 'Expired':
                  # Display a warning alert in red color
                  messagebox.showwarning("Warning Alert", "The license for this vehicle has expired!", icon="warning")
              else:
                   # Display a message indicating that the license is not expired
                   messagebox.showinfo("Valid", "The license for this vehicle is valid.")

              # Display the row as a table
              row_index = 0
              for key, value in row.items():
                    # Create a label for the key and place it in the table frame
                    label_key = tk.Label(table_frame, text=key, font=("Arial", 10, "bold"))
                    label_key.grid(row=row_index, column=0)
        
                    # Create a label for the value and place it in the table frame
                    label_value = tk.Label(table_frame, text=value, font=("Arial", 10))
                    label_value.grid(row=row_index, column=1)
        
                    row_index += 1
        
                    # Configure the columns of the table frame to have equal weight
                    table_frame.columnconfigure(0, weight=1)
                    table_frame.columnconfigure(1, weight=1)
           else:
              # If no row is found for the license plate number, display a label indicating that the license plate number was not found in the database
              not_found_label = tk.Label(window3, text="License plate not found", font=("Arial", 12))
              not_found_label.pack()


        #New function for creating third GUI page for displaying owner Information that are retrieved by Integrating the ALPR with database :Integrated Database
        #open_window3()function will be executed when the "Check" button is clicked. 
        def open_window4(license_plate, info_type):
           ## Create a new window for displaying information
           window3 = tk.Toplevel()
           window3.title("Integrated Database")
           window3.geometry("800x600")
           
           # Retrieve the owner information from the excel sheet file by calling (search_owner_information function) which takes (license_plate) number that was entered by the user
           row = search_owner_information(license_plate)
           if row:
              # Create a label for displaying information
              info_label = tk.Label(window3, text="Owner's Information", font=("Arial", 12))
              info_label.pack()

              # Create a frame for displaying the information in a table format
              table_frame = tk.Frame(window3)
              table_frame.pack()

              # Display the row as a table
              row_index = 0
              for key, value in row.items():
                    # Create a label for the key and place it in the table frame
                    label_key = tk.Label(table_frame, text=key, font=("Arial", 10, "bold"))
                    label_key.grid(row=row_index, column=0)
        
                    # Create a label for the value and place it in the table frame
                    label_value = tk.Label(table_frame, text=value, font=("Arial", 10))
                    label_value.grid(row=row_index, column=1)
        
                    row_index += 1
        
                    # Configure the columns of the table frame to have equal weight
                    table_frame.columnconfigure(0, weight=1)
                    table_frame.columnconfigure(1, weight=1)
           else:
              # If no row is found for the license plate number, display a label indicating that the license plate number was not found in the database
              not_found_label = tk.Label(window3, text="License plate not found", font=("Arial", 12))
              not_found_label.pack()      



        # Create three white square windows with titles to display the detected and cropped license plate images
        label1 = tk.Label(window2, text="Image 1", font=("Arial", 16))
        label1.grid(row=1, column=1, padx=5, pady=5)
        panel1 = tk.Label(window2, bg="white", width=40, height=20)
        panel1.grid(row=2, column=1, padx=5, pady=5)

        label2 = tk.Label(window2, text="Image 2", font=("Arial", 16))
        label2.grid(row=1, column=2, padx=5, pady=5)
        panel2 = tk.Label(window2, bg="white", width=40, height=20)
        panel2.grid(row=2, column=2, padx=5, pady=5)

        label3 = tk.Label(window2, text="Image 3", font=("Arial", 16))
        label3.grid(row=1, column=3, padx=5, pady=5)
        panel3 = tk.Label(window2, bg="white", width=40, height=20)
        panel3.grid(row=2, column=3, padx=5, pady=5)

        label4 = tk.Label(window2, text="Digital Text", font=("Arial", 16))
        label4.grid(row=1, column=4, padx=5, pady=5)
        panel4 = tk.Label(window2, bg="white", width=30, height=20)
        panel4.grid(row=2, column=4, padx=5, pady=5)


        # Start_license_plate_detection function for starting the detection model, and character recognition of detected license plate
        # The start_license_plate_detection()function will be executed when the "Start" button is clicked. 
        def start_license_plate_detection():
             
             # Specify the image folder path, configuration path, and weights path
             folder_path = 'C:\\Users\\Home\\Desktop\\yolov5test\\ALPR_Model\\Input_camera_images'
             config_path = 'C:\\Users\\Home\\Desktop\\yolov5test\\yolov5\\data\\coco128.yaml'
             weights_path = 'C:\\Users\\Home\\Desktop\\yolov5test\\best8.pt'
              
             # Read images from the specified folder path by calling the read_images_from_folder function 
             images = read_images_from_folder(folder_path)
             panels = [panel1, panel2, panel3]
             ocr_results = []

             for i in range(len(images)):
                  # Process each image for license plate detection
                  image_path = images[i]
                  # Calling the detect_license_plate function for detecting the LP from image and save it in cropped_image variable
                  cropped_image = detect_license_plate(image_path, config_path, weights_path)
                  #Resizing the cropped_image in order to fit within the size of the white window
                  resized_image = cropped_image.resize((40, 20), Image.ANTIALIAS)
                  tk_image = ImageTk.PhotoImage(resized_image)

                  # Update the panel and OCR result based on the index i
                  panels[i].configure(image=tk_image)
                  panels[i].image = tk_image
                   # Save the cropped ith different namesmage wi
                  cropped_image.save(f'cropped{i+1}.jpg')
                  # Call OCR_Character_recognition function with the cropped image
                  ocr_result = OCR_Character_recognition(cropped_image)
                  ocr_results.append(ocr_result)

             # Concatenate the OCR results with newline character
             OCR_results_text = '\n'.join(ocr_results)
             panel4.configure(text=OCR_results_text)

        # Create a start button, which when clicked it intialize the detection model to start detecting 
        # by executing the start_license_plate_detection() function  
        start_button = tk.Button(window2, bg="green", text="Start", width=10, font=("Arial", 12), command=start_license_plate_detection)
        start_button.grid(row=3, column=1, padx=5, pady=5)

        # Create a selection box with a title and two options
        option_label = tk.Label(window2, text="Selection Box",font=("Arial",12))
        option_label.grid(row=3, column=2, padx=5,pady=5)
        options = ["Retrieve Owner's Information", "Retrieve Vehicle's Information"]
        selection_variable = tk.StringVar()
        selection_box = tk.OptionMenu(window2, selection_variable, *options)
        selection_box.grid(row=4, column=2, padx=5, pady=5)
        
        #focus_input_field function will be executed when "Enter License Plate" is clicked to enter the LP number
        def focus_input_field():
              # Function to set focus on the input field
              input_field.focus()

        # Create a button labeled "Enter License Plate"
        # The button calls the "focus_input_field()" function when clicked
        button = tk.Button(window2,text="Enter License Plate",width=20,font=("Arial",12) ,command=focus_input_field)
        button.grid(row=4, column=1, padx=5, pady=5)
        # Create an input field for the license plate entry with a white background 
        input_field = tk.Entry(window2, bg="white", width=30)
        input_field.grid(row=5, column=2, padx=5, pady=5)

         # check_button_clicked () function will be executed when the "check" button is clicked
        def check_button_clicked():
             # Function to handle the click event of the "Check" button
             license_plate = input_field.get() # Get the license plate entered in the input field
             info_type = selection_variable.get() # Get the selected information type
             if info_type == "Retrieve Vehicle's Information":
                 open_window3(license_plate, info_type)  # Open window3 with the provided license plate and info type
             else:
                 open_window4(license_plate,info_type)
       
        # Create a button labeled "Check" , the button calls the "check_button_clicked()" function when clicked.
        check_button = tk.Button(window2, text="Check", width=10, font=("Arial", 12), command=check_button_clicked)
        check_button.grid(row=4, column=3, padx=5, pady=5)

        # Function to stop the license plate detection process, it will be executed when the "Stop" button is clicked
        def stop_detection():
            #window2.destroy() # Close window2 (commented out)
            panel1.configure(image="") # Clear the image displayed in panel1
            panel2.configure(image="") # Clear the image displayed in panel2
            panel3.configure(image="") # Clear the image displayed in panel3
            panel4.configure(text="")
        
        # Create a button labeled "Pause"
        pause_button = tk.Button(window2, bg="green",text="Pause", width=10,font=("Arial",12))
        pause_button.grid(row=3, column=3, padx=10, pady=10)

        #Create a button labeled "Stop", the button calls the "stop_detection()" function when clicked
        stop_button = tk.Button(window2, bg="red",text="Stop", width=10,font=("Arial",12),command=stop_detection)
        stop_button.grid(row=5, column=3, padx=10, pady=10)

    else:
        # if the username and password are incorrect, show a message box with an error message
        messagebox.showerror("Login Error", "Invalid username or password")


# creating the main window for the ALPR system :login page 
window = tk.Tk()
window.title("ALPR Login")
window.geometry("600x600")

# create a frame for the username and password labels and entry fields
frame_login = tk.Frame(window, padx=10, pady=10)

# create the username label and entry field
label_username = tk.Label(frame_login, text="Username", font=("Arial", 16))
label_username.grid(row=1, column=0)
entry_username = tk.Entry(frame_login, font=("Arial", 16))
entry_username.grid(row=1, column=1, pady=20)

# create the password label and entry field
label_password = tk.Label(frame_login, text="Password", font=("Arial", 16))
label_password.grid(row=2, column=0)
entry_password = tk.Entry(frame_login, show="*", font=("Arial", 16))
entry_password.grid(row=2, column=1, pady=20)

# create a button to check the login credentials, the check_login() function will be executed when the user click "login" button
# where the second GUI window/page will appear
button_login = tk.Button(window, text="Login", bg='#666666', fg='#FFFFFF', font=("Arial", 16), command=check_login)
# pack the widgets onto the window
frame_login.pack()
button_login.pack()

# start the GUI
window.mainloop()